In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#import library
import tensorflow as tf
import zipfile,os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from google.colab import files
from tensorflow.keras.preprocessing import image


In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

# Extract the zip file
local_zip = '/content/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
# View folder name in zip file
view_dir = '/content/rockpaperscissors/rps-cv-images'
print("Names of the folders inside the zip file : ", os.listdir(view_dir))

In [ ]:
# Defining the rock, paper, and scissors image training folder
fold_rock = os.path.join('/content/rockpaperscissors/rock')
fold_paper = os.path.join('/content/rockpaperscissors/paper')
fold_scissors = os.path.join('/content/rockpaperscissors/scissors')

In [ ]:
# jumlah gambar batu
print("Jumlah gambar batu :",len(os.listdir(fold_rock)))
# Jumlah gambar kertas
print("Jumlah gambar kertas :",len(os.listdir(fold_paper)))
# Jumlah gambar gunting
print("Jumlah gambar gunting :",len(os.listdir(fold_scissors)))

In [ ]:
# Defining the main folder
db_dir = view_dir

In [ ]:
# Grouping data into Training Data and Validation Data
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split=0.4)

In [ ]:
# Data labeling using ImageDataGenerator by folder
train_generator = train_datagen.flow_from_directory(
        db_dir,
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        # karena ini merupakan masalah klasifikasi 3 kelas, gunakan class_mode = 'categorical'
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        db_dir,
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
       # karena ini merupakan masalah klasifikasi 3 kelas, gunakan class_mode = 'categorical'
        class_mode='categorical',
        shuffle=True,
        subset='validation')

In [ ]:
# penggunaan Callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>=0.93):
      print("\nAkurasi di atas 90%, hentikan training")
      self.model.stop_training= True
callbacks = myCallback()

In [ ]:
# Building an artificial neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy'
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# latih model dengan model.fit
history = model.fit(
      train_generator,
      steps_per_epoch=25,  # batch yang akan dieksekusi setiap epoch
      epochs=30, #penambahan epochapabila akurasi belum maksimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data
      validation_steps=5,  # batch yang akan dieksekuasi setiap epoch
      verbose=2, callbacks=[callbacks])

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))

  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)
  print(fn)
  if classes[0][0]==1:
    print('Gambar kertas')
  elif classes[0][1]==1:
    print('Gambar batu')
  elif classes[0][2]==1:
    print('Gambar gunting')
  else:
   print('Unknown')

In [ ]:
classes

In [ ]:
print(train_generator.class_indices)